## 設計日期迴圈
- PTT網頁版1頁有20篇文章
- PTT會固定刪文章，約停在15000頁
- 約前兩天的文章會留下最多

In [2]:
# #抓日期
# import time
# import datetime
# #今天日期 
# today = datetime.date.today()
# print "今天日期 =>" + str(today)
#設定要相減的日期
# other_day = datetime.date(2016,9,6)
# result = today - other_day
# print result
# #如果只想要把相差的天數拿出來的話
# day = result.days
# print "相減後的天數 =>" + str(day)

今天日期 =>2016-09-19


## Step1 設定抓取頁數
- 先抓到總頁數
- 往前推150頁抓取

In [22]:
#coding=utf-8 
import requests
from bs4 import BeautifulSoup
#通過18禁頁面
url = 'https://www.ptt.cc/bbs/Gossiping/index.html'
playload = {
    'from':'/bbs/Gossiping/index.html',
    'yes':'yes'
}
rs = requests.session() #設定一個session
res1 = rs.post('https://www.ptt.cc/ask/over18',verify=False,data = playload) #透過session記錄通過18禁頁面
res = rs.get(url,verify=False)
soup = BeautifulSoup(res.text)
temp_page_num = soup.select('.btn.wide')[1]['href'].split('index')[1].split('.')[0]
page_num = int(temp_page_num)+1
# print '總頁數'+str(page_num)

In [24]:
#每天要爬取多少頁數
num = 150

#啟始頁面index
starIndex = page_num-num

#結束頁面index
endIndex = starIndex + num

print starIndex
print endIndex

14912
15062


## Step2 爬取前一天的PTT文章

In [26]:
#抓日期
import time
import datetime
today = datetime.datetime.now()
#前一天的日期
yestoday = today + datetime.timedelta(days = -1)
yestoday = yestoday.strftime('%Y%m%d')
fileName = yestoday
year =  today.strftime('%Y')
print yestoday

20160925


## Step3 爬取每一頁的文章url

In [50]:
import requests
from bs4 import BeautifulSoup
from urlparse import urljoin
import time
import datetime

#主要網址
main_url = 'https://www.ptt.cc/bbs/Gossiping/'
errorCount = 0
error_url = ''

#要印出的 url 長串
urlString = ''

#相關資料
playload = {
    'from':'/bbs/Gossiping/index.html',
    'yes':'yes'
}
#設定一個session
rs = requests.session()

#先session記錄通過18禁頁面
res1 = rs.post('https://www.ptt.cc/ask/over18',verify=False,data = playload)


#開始爬文
for n in range(starIndex,endIndex):
    url = 'https://www.ptt.cc/bbs/Gossiping/index{}.html'.format(n)

    #進入爬取內文
    res = rs.get(url,verify=False)
    soup = BeautifulSoup(res.text)
    try:
        for entry in soup.select('.r-ent'):
        #     print len(entry.select('.title a'))

            #判斷該文章是否已被刪除，刪除即"跳過"
            if len(entry.select('.title a ')) != 0 :

                #爬取資訊
                nrec = entry.select('.nrec')[0].text.strip()
                if nrec=='':
                    nrec='0'
                title = entry.select('.title')[0].text.strip()
                category = category = title.split('[')[1].split(']')[0].strip()
                author = entry.select('.author')[0].text.strip()
                href =  entry.select('a')[0]['href']
                article_url = urljoin(main_url,href) #urljoin會自動合併兩個網址

                #設定日期
                date = entry.select('.date')[0].text.strip()
                month,day = date.split('/')[0].strip(),date.split('/')[1].strip()
                if int(month) < 10:
                    month = '0'+month
                newDate = year+month+day
                
                #文章url為前兩天日期，才寫入file
                date_time_1 = datetime.datetime.strptime(newDate,'%Y%m%d')
                date_time_2 = datetime.datetime.strptime(yestoday,'%Y%m%d')
                time.sleep(1)
                
                if date_time_1 == date_time_2:
                    urlString += article_url+','+category+','+nrec+','+title+','+newDate+','+author+'\n'

        #         print article_url,title,category,newDate,author,nrec
        time.sleep(4)
    except:
        errorCount += 1
        error_url += article_url+','
        continue
# print urlString
f = open('C:/Users/ytchen/Desktop/ptt/{}.txt'.format(fileName),'w')
f.write(urlString.encode('utf-8')) #最後再一次寫出來
f.close()   

#log記錄
logString = ''
fileName_log = yestoday+'/'+str(starIndex)+'/'+str(endIndex)+'/'+'error:'+str(errorCount)+'\n'+'error_url:'+error_url+'\n'
f1 = open('C:/Users/ytchen/Desktop/ptt/log.txt','r')
for logLine in f1.readlines():
    logString += logLine
f1.close()

f2 = open('C:/Users/ytchen/Desktop/ptt/log.txt','w')
logString += fileName_log
f2.write(logString.encode('utf-8'))
f2.close() 